# Выделение терминов

In [1]:
! export PATH=$PATH:/home/jupyter/.local/bin

In [16]:
%pip install --upgrade requests botocore urllib3 --user

In [17]:
%pip install pymorphy2

In [18]:
%pip install git+https://github.com/openai/whisper.git

In [19]:
%pip install peft

In [20]:
%pip install --no-cache-dir transformers sentencepiece

In [ ]:
%pip install python-docx

In [23]:
%pip freeze > pip_freeze.txt

In [37]:
%zip defining_utils.py

UsageError: Line magic function `%zip` not found.


In [36]:
! ls *.py 

defining_utils.py
main.py
ner_utils.py
prepare_for_docx.py
summarization_utils.py
transcript_utils.py
write_docx.py


In [15]:
## тестим

! python3 main.py

WHISPER IS INITIALIZED
SAIGA IS INITIALIZED
TRANSCRIPT IS DONE
TERMS ARE DONE
DEFS: ['Бизнес - это деятельность, направленная на получение прибыли путем продажи товаров или услуг. Это может быть маленький магазин, большая компания или даже интернет-проект. Бизнес может быть различных видов, например, торговый, производственный, финансовый или услуги.', 'Факторы - это различные влияющие на результат или процесс. В контексте стратегического планирования и управления бизнесом, факторы могут быть разделены на внешние (экстернальные) и внутренние (интернальные).', 'Маркетинг - это система методов и техник, используемых для изучения, планирования, разработки, продвижения и продажи товаров, услуг и идей. Маркетинговые методы включают исследование рынка, определение целевой аудитории, разработку стратегии продаж, создание бренда, разработку рекламных кампаний и многое другое.', 'Эффективность - это способность или степень того, насколько хорошо и успешно ресурсы (включая время, деньги, усилия 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
 99%|█████████▉| 363777/366777 [05:26<00:02, 1114.42frames/s]


## Импорты

In [7]:
import nltk
from nltk.corpus import stopwords
import pymorphy2
import re
from collections import Counter
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import whisper


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## NER (rule-based) utils

In [8]:
ADDITIONAL_STOPWORDS = [
    'a', 'about', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be',
    'been', 'but', 'by', 'can', 'could', 'do', 'for', 'from', 'has', 'have',
    'i', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'not', 'of', 'on', 'one',
    'or', 'so', 'that', 'the', 'them', 'there', 'they', 'this', 'to', 'was',
    'we', 'what', 'which', 'will', 'with', 'would', 'you', 'а', 'будем', 'будет',
    'будете', 'будешь', 'буду', 'будут', 'будучи', 'будь', 'будьте', 'бы', 'был',
    'была', 'были', 'было', 'быть', 'в', 'вам', 'вами', 'вас', 'весь', 'во', 'вот',
    'все', 'всё', 'всего', 'всей', 'всем', 'всём', 'всеми', 'всему', 'всех', 'всею',
    'всея', 'всю', 'вся', 'вы', 'да', 'для', 'до', 'его', 'едим', 'едят', 'ее',
    'её', 'ей', 'ел', 'ела', 'ем', 'ему', 'емъ', 'если', 'ест', 'есть', 'ешь',
    'еще', 'ещё', 'ею', 'же', 'за', 'и', 'из', 'или', 'им', 'ими', 'имъ', 'их',
    'к', 'как', 'кем', 'ко', 'когда', 'кого', 'ком', 'кому', 'комья', 'которая',
    'которого', 'которое', 'которой', 'котором', 'которому', 'которою', 'которую',
    'которые', 'который', 'которым', 'которыми', 'которых', 'кто', 'меня', 'мне',
    'мной', 'мною', 'мог', 'моги', 'могите', 'могла', 'могли', 'могло', 'могу',
    'могут', 'мое', 'моё', 'моего', 'моей', 'моем', 'моём', 'моему', 'моею',
    'можем', 'может', 'можете', 'можешь', 'мои', 'мой', 'моим', 'моими', 'моих',
    'мочь', 'мою', 'моя', 'мы', 'на', 'нам', 'нами', 'нас', 'наса', 'наш', 'наша',
    'наше', 'нашего', 'нашей', 'нашем', 'нашему', 'нашею', 'наши', 'нашим', 'нашими',
    'наших', 'нашу', 'не', 'него', 'нее', 'неё', 'ней', 'нем', 'нём', 'нему', 'нет',
    'нею', 'ним', 'ними', 'них', 'но', 'о', 'об', 'один', 'одна', 'одни', 'одним',
    'одними', 'одних', 'одно', 'одного', 'одной', 'одном', 'одному', 'одною', 'одну',
    'он', 'она', 'оне', 'они', 'оно', 'от', 'по', 'при', 'с', 'сам', 'сама', 'сами',
    'самим', 'самими', 'самих', 'само', 'самого', 'самом', 'самому', 'саму', 'свое',
    'своё', 'своего', 'своей', 'своем', 'своём', 'своему', 'своею', 'свои', 'свой',
    'своим', 'своими', 'своих', 'свою', 'своя', 'себе', 'себя', 'собой', 'собою',
    'та', 'так', 'такая', 'такие', 'таким', 'такими', 'таких', 'такого', 'такое',
    'такой', 'таком', 'такому', 'такою', 'такую', 'те', 'тебе', 'тебя', 'тем',
    'теми', 'тех', 'то', 'тобой', 'тобою', 'того', 'той', 'только', 'том', 'томах',
    'тому', 'тот', 'тою', 'ту', 'ты', 'у', 'уже', 'чего', 'чем', 'чём', 'чему',
    'что', 'чтобы', 'эта', 'эти', 'этим', 'этими', 'этих', 'это', 'этого', 'этой',
    'этом', 'этому', 'этот', 'этою', 'эту', 'я', 'мені', 'наші', 'нашої', 'нашій',
    'нашою', 'нашім', 'ті', 'тієї', 'тією', 'тії', 'теє', 'очень', 'вновь', 'казаться',
    'nan', 'всё', 'это', 'твой', 'свой', 'весь', 'вообще',
    'пусть', 'ещё', 'который', 'её', 'словно', 'пока',
    'сколько', 'сквозь', 'чей', 'вокруг', 'любой', 'слишком', 'почему',
    'среди', 'значит', 'ваш', 'тысяча', 'однажды', 'сегодня',
    'завтра', 'вчера', 'послезавтра', 'позавчера', 'нибыть', 'точно',
    'уметь', 'целый', 'полный', 'часть', 'делать',
    'готовый', 'хотя', 'как', 'столько', 'равно', 'оно', 'похожий',
    'скоро', 'разный', 'всякий', 'порой', 'часто', 'настоящий', 'вместо',
    'вовсе', 'иметь', 'вроде', 'лишь', 'также', 'млн', 'руб',
    'наверное', 'три', 'четыре', 'пять', 'шесть', 'семь', 'восемь',
    'девять', 'десять', 'пускай', 'мимо', 'частый', 'сей', 'привет',
    'наверно', 'туда', 'иль', 'возможно', 'вено', 'иной', 'едва',
    'поэтому', 'впереди', 'зря', 'вообще', 'меж', 'мол', 'нету', 'наш',
    'иначе', 'всякий', 'ради', 'оттого', 'нечего', 'скорее',
    'пред', 'сначала', 'вряд', 'случайно', 'коль', 'многий', 'кроме',
    'откуда', 'нынче', 'ибо', 'порою', 'вне', 'кой', 'еле', 'разом',
    'либо', 'зато', 'сорок', 'везде', 'таков', 'вон', 'покуда',
    'затем', 'особенно', 'несмотря', 'сотый', 'всюду',
    'возле', 'кстати', 'прям', 'ныне', 'бай', 'предельно',
    'чрез', 'некий', 'впредь', 'практически', 'лишь', 'каждый',
    'день', 'год', 'мочь', 'коли', 'найти', 'нужно', 'снова',
]

EDUCATIONAL_STOPWORDS = [
    "друг", "коллега", "блок", "лекция", "семинар", "вебинар", "урок", "курс",
    "модуль", "тренинг", "практикум", "мастер-класс", "туториал", "студент",
    "преподаватель", "лектор", "учитель", "обучающийся",
    "группа", "класс", "аудитория", "куратор", "задание", "проект",
    "домашний", "работа", "экзамен", "тест", "оценка", "дедлайн", "кредит",
    "аттестация", "сертификация", "учебный", "план", "образовательная",
    "программа", "академическая степень", "научное", "исследование", "материал",
    "методические указания", "учебник", "литература", "конспект", "основы",
    "понимание", "случай", "идея", "задание", "тема", "первый", "второй",
    "практика", "чиселка", "лайфхак", "понятие", "плюсик", "профессия"
]

GARBAGE = [
    "внимание", "штука", "текст", "текст", "страничка", "color", "центр",
    "доллар", "стиль", "отступ", "файл", "экран", "пример", "hello", "значение",
    "этап"
]

gb_stopwords = stopwords.words('russian') + stopwords.words('english') + ADDITIONAL_STOPWORDS + EDUCATIONAL_STOPWORDS + GARBAGE

In [9]:
morph = pymorphy2.MorphAnalyzer()

# правило — последовательность слов (список) с их частями речи и падежами
morph_rules = [
    [{"POS": "NOUN", "case": ["nomn", "accs"]}],
    [{"POS": "ADJF", "case": ["nomn", "accs"]}, {"POS": "NOUN", "case": ["nomn", "accs"]}],
    [{"POS": "NOUN", "case": ["nomn", "accs"]}, {"POS": "NOUN", "case": ["gent"]}],
    [{"POS": "NOUN", "case": ["nomn", "accs"]}, {"POS": "ADJF", "case": ["gent"]}, {"POS": "NOUN", "case": ["gent"]}],
    [{"POS": "ADJF", "case": ["nomn", "accs"]}, {"POS": "ADJF", "case": ["nomn", "accs"]}, {"POS": "NOUN", "case": ["nomn", "accs"]}],
    [{"POS": "ADJF", "case": ["nomn", "accs"]}, {"POS": "NOUN", "case": ["nomn", "accs"]}, {"POS": "NOUN", "case": ["gent"]}]
]

english_rule = re.compile("[A-Za-z\d-]{3,}(?:\s+[A-Za-z\d-]{3,})*")

def match_morph_rule(text, rule):
    result = False
    words = text.split()
    if len(words) != len(rule):
        return False
    flag = False
    for word, rule_part in zip(words, rule):
        parsed = morph.parse(word)[0]
        if parsed.tag.POS != rule_part["POS"]:
            flag = True
        if parsed.tag.case not in rule_part["case"]:
            flag = True
    if not flag:
        result = True
    return result

def exclude_stopwords(tokens):
    return [word for word in tokens if word.lower() not in gb_stopwords]

In [10]:
def preprocess(text, lowercase=True):
    text = text.replace("\n", "")
    text = re.sub(r"( - |- )", r" ", text)
    text = re.sub(r"\b(\w+)( \1\b)+", r"\1", text)
    text = re.sub(r"([0-9]+).([0-9]+)", "", text)
    text = re.sub(r"([0-9]+).(\s+)", "", text)
    text = re.sub(r"[:\.,»«)(\n\t–]", ". ", text)
    text = re.sub(r"([а-яa-zё]+)([А-ЯA-ZЁ]+)", r"\1. \2", text)
    text = re.sub(r"([0-9]+)(\s+)([0-9]+)", "", text)
    text = re.sub(r"\.(\s+)([a-zA-Zа-яА-ЯёЁ0-9\-])", lambda x: x.group(0).upper(), text)
    text = text.replace("\xa0", " ")
    text = text.replace(".", ". ").strip()
    if lowercase:
        return re.findall(r'\b\w+\b', text.lower())
    return re.findall(r'\b\w+\b', text)

def find_english_terms(text):
    return list(english_rule.findall(text))

def find_morph_terms(text, stopwords=False):
    terms = []
    tokens = preprocess(text)
    if stopwords:
        tokens = exclude_stopwords(tokens)

    for i in range(len(tokens)):
        eng_result = find_english_terms(' '.join(tokens[i:i+3]))
        if eng_result:
            terms.extend(eng_result)
        for rule in morph_rules:
            phrase = ' '.join(tokens[i:i+len(rule)])
            if match_morph_rule(phrase, rule):
                terms.append(phrase)

    return terms

In [11]:
# strong: если хотя бы один из токенов термина — стопслово, то удаляем термин
# иначе удаляем только токен, оставляем остаток термина
def filter_terms(terms, strong=False):
    filtered_terms = []

    for term in terms:
        words = term.split()
        normalized_words = [morph.parse(word)[0].normal_form for word in words]
        if strong:
            if not any(word in gb_stopwords for word in normalized_words):
                filtered_terms.append(term)
        else:
            result = ' '.join([words[i] for i in range(len(words)) if normalized_words[i] not in gb_stopwords])
            if result != "":
                filtered_terms.append(result)

    return filtered_terms

def get_terms_candidates(text, strong=False):
    terms = find_morph_terms(text)

    return filter_terms(list(terms), strong=strong)

In [12]:
def normalize_terms(words):
    return [morph.parse(word)[0].normal_form for word in words]

def normalize_frequencies(frequencies):
    max_freq = max(frequencies.values())
    return {term: freq / max_freq for term, freq in frequencies.items()}

def filter_terms_by_frequency(terms, threshold=0.15):
    term_frequencies = Counter(terms)
    normalized_frequencies = normalize_frequencies(term_frequencies)
    filtered_terms = {term: freq for term, freq in normalized_frequencies.items() if freq >= threshold}
    return filtered_terms

def finalize_terms(terms, filtered_terms):
    return [term for term in terms if term in filtered_terms.keys()]

## Whisper + DEVICE

In [13]:
# init whisper
model_size = "small"

# define device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# load model
whisper_model = whisper.load_model(model_size)
whisper_model.to(DEVICE)

100%|███████████████████████████████████████| 461M/461M [00:10<00:00, 45.5MiB/s]


Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-11): 12 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=False)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (out): Linear(in_features=768, out_features=768, bias=True)
        )
        (attn_ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (mlp_ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-0

In [14]:
DEVICE

'cuda'

In [15]:
def whisper_transcribe_file(model, path_to_mp3_file):
    transcription = model.transcribe(path_to_mp3_file, language="ru", verbose=False)

    return transcription

In [16]:
# path_to_mp3 = 'drive/MyDrive/ИТМО/Хак/audio2.mp3'
# transcript = whisper_transcribe_file(model, path_to_mp3)

# lecture_text = transcript['text']

# terms = get_terms_candidates(lecture_text, strong=True)
# cleared_terms = filter_terms_by_frequency(terms)
# final_terms = finalize_terms(terms, cleared_terms)
# final_terms_unique = cleared_terms.keys()

In [17]:
def extract_timecodes_and_context(transcript, terms):
    segments = transcript['segments']
    timecodes = dict()
    context = dict()

    for term in terms:
        timecodes[term] = [(segment["start"], segment["end"]) for segment in segments if term in segment['text'].strip().lower()]
        context[term] = [segment["text"] for segment in segments if term in segment['text'].strip().lower()]

    return timecodes, context

In [18]:
# timecodes, context = extract_timecodes_and_context(transcript, final_terms_unique)

# Генерация определений

In [19]:
adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"

tokenizer = AutoTokenizer.from_pretrained(
              base_model_name,
              trust_remote_code=True,
              use_fast=False)

tokenizer.pad_token = tokenizer.eos_token
device_map = {"": DEVICE}

model = AutoPeftModelForCausalLM.from_pretrained(
              adapt_model_name,
              device_map=device_map,
              torch_dtype=torch.bfloat16)

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

In [20]:
model.device

device(type='cuda', index=0)

In [21]:
def get_answer(model, tokenizer, prompt, DEVICE):
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    outputs = model.generate(input_ids=inputs["input_ids"].to(DEVICE),
                            top_p=0.5,
                            temperature=0.3,
                            attention_mask=inputs["attention_mask"],
                            max_new_tokens=128,
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True)

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    parsed_answer = output.split("Вот ответ на ваш вопрос:")[1].strip()

    if "bot:" in parsed_answer:
        parsed_answer = parsed_answer.split("bot:")[0].strip()


    if not parsed_answer.endswith('.'):
        parsed_answer_sentences = parsed_answer.split('.')
        parsed_answer = '.'.join(parsed_answer_sentences[:-1]) + '.'


    parsed_answer = parsed_answer.replace('</s>', '')
    parsed_answer = parsed_answer.replace('bot', '')

    return parsed_answer

In [22]:
rag_prompt = '''<s>system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s>
<s>user
Текст: {}
Вопрос: что такое {}?</s>
<s>bot
Вот ответ на ваш вопрос:'''

In [23]:
def get_rag_context(term_fragments, CONTEXT_WORDS_THRESH=50):
    context = ""

    for fragment in term_fragments:
        if len(context.split(' ')) + len(fragment.split(' ')) < CONTEXT_WORDS_THRESH:
            context += fragment

    context = context.strip()

    return context

In [24]:
def get_rag_prompts(terms_and_context_dict):
    rag_prompts = []

    for term_to_define, term_fragments in terms_and_context_dict.items():
        term_context = get_rag_context(term_fragments)
        prompt = rag_prompt.format(term_context, term_to_define)

        rag_prompts.append(prompt)

    return rag_prompts

## Пайплайн:

1. Транскрибируем аудио

2. Выделение терминов

3. Из первого словаря делает submission файл

4. Из второго словаря генерируем определения

5. (?) Делаем суммаризацию по фрагментам, в которых встречались термины

In [25]:
paths = [
    "/home/jupyter/datasphere/project/audio5.mp3",
    "/home/jupyter/datasphere/project/audio6.mp3"
]

In [ ]:
transcript = whisper_transcribe_file(whisper_model, paths[0])

 59%|█████▊    | 229000/390421 [04:40<02:55, 917.30frames/s] 

In [ ]:
lecture_text = transcript['text']

In [ ]:
terms = get_terms_candidates(lecture_text, strong=True)
cleared_terms = filter_terms_by_frequency(terms)
final_terms = finalize_terms(terms, cleared_terms)
final_terms_unique = cleared_terms.keys()

In [ ]:
timecodes, context = extract_timecodes_and_context(transcript, final_terms_unique)

In [ ]:
definitions = []

for prompt in get_rag_prompts(context):
    definitions.append(get_answer(model, tokenizer, prompt, DEVICE))

In [ ]:
# timecodes

In [ ]:
# context

In [ ]:
# definitions

In [ ]:
# ! python3 transcript_utils.py

In [ ]:
term_definition_timecodestart_timecodeend = []

for i, para in enumerate(timecodes.items()):
    term, term_timecodes = para
    definition = definitions[i]
    for term_timecode in term_timecodes:
        term_definition_timecodestart_timecodeend.append((paths[0].split('/')[-1], term, definition, term_timecode[0], term_timecode[1]))

In [ ]:
import pandas as pd

overall_df = pd.DataFrame(term_definition_timecodestart_timecodeend, columns=['File', 'Term', 'Definition', 'Time_start', 'Time_end'])

In [ ]:
our_cringe_df = overall_df.drop_duplicates(subset=['File', 'Term']) 
our_cringe_df

In [ ]:
f_t_d_s_e = list(our_cringe_df.to_records(index=False))

In [ ]:
from datetime import timedelta

def convert_float_to_hh_mm_ss(seconds_float):
    delta = timedelta(seconds=seconds_float)
    hours, remainder = divmod(delta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    return f"{hours:02}:{minutes:02}:{seconds:02}"

In [ ]:
csv_file_term = overall_df[['File', 'Term']].drop_duplicates()
csv_file_term.head()

In [ ]:
csv_file_term_timecode = overall_df[['File', 'Term', 'Time_start', 'Time_end']].drop_duplicates()
csv_file_term_timecode['Time_start'] = csv_file_term_timecode['Time_start'].apply(convert_float_to_hh_mm_ss)
csv_file_term_timecode['Time_end'] = csv_file_term_timecode['Time_end'].apply(convert_float_to_hh_mm_ss)
csv_file_term_timecode['Time'] = csv_file_term_timecode['Time_start'] + ' - ' + csv_file_term_timecode['Time_end']
csv_file_term_timecode.drop(columns=['Time_start', 'Time_end'], inplace=True)
csv_file_term_timecode.head()

## Для суммаризации

In [ ]:
def create_5mins_batches(segments, batch_time_s=300):
    batch_and_start = []
    
    cur_time = 0 
    cur_start = 0
    cur_batch = ""
    
    for segment in segments:
        seg_text = segment['text']
        seg_start = segment['start']
        seg_fin = segment['end']
        
        seg_time = seg_fin - seg_start
        if cur_time + seg_time <= batch_time_s:
            cur_batch += seg_text
            cur_time += seg_time
        
        else:
            batch_and_start.append(tuple([cur_batch.strip(), cur_start, cur_start + cur_time]))
            cur_start += cur_time 
            cur_time = 0
            cur_batch = ""
            
    if cur_batch != "":
        batch_and_start.append(tuple([cur_batch.strip(), cur_start, cur_start + cur_time]))
            
    return batch_and_start
            

In [ ]:
batches = create_5mins_batches(transcript['segments'])

In [ ]:
# batches

In [ ]:
summary_prompt = ''' 
    <s>system 
    Ты — русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им извлекать нужную информацию из текста.</s> 
    <s>user 
    Текст: {} 
    Вопрос: о чем рассказывается в данной части лекции? Какие темы рассмотрены? Ответь одним предложением.</s> 
    <s>bot 
    Вот ответ на ваш вопрос:'''

In [ ]:
summary_list = []

for batch in batches:
    batch_text, batch_start, batch_end = batch[0], batch[1], batch[2]
    batch_prompt = summary_prompt.format(batch_text)
    batch_summary = get_answer(model, tokenizer, batch_prompt, DEVICE)
    
    # пост-обработка
    batch_summary = batch_summary.split('\n')[0].capitalize()
    
    summary_list.append([batch_summary, batch_start, batch_end])

In [ ]:
len(summary_list)

## Сводим воедино для docx

In [ ]:
%pip install python-docx

In [ ]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

from datetime import timedelta

In [ ]:
# это что за ужас?

overall_list_of_tuples = [(None, None, summary[0], summary[1], summary[2]) for summary in summary_list] + f_t_d_s_e

In [ ]:
overall_list_of_tuples = sorted(overall_list_of_tuples, key=lambda tup: float(tup[3]))

In [ ]:
list_items_for_docx = []

for item in overall_list_of_tuples:
    if not item[0]:
        # почему-то без норм предобработки
        summary = item[2].split('\n')[0].capitalize()
        list_items_for_docx.append(summary)
    else:
        list_items_for_docx.append(item)

In [ ]:
list_items_for_docx[0]

In [ ]:
def convert_to_hh_mm_ss(time_float):
    # Convert time in float format to hh:mm:ss string
    total_seconds = int(time_float)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)

    return f"{hours:02}:{minutes:02}:{seconds:02}"

# не добавили тамйкоды для саммари
# отступы странные
def create_docx_from_list(data):
    doc = Document()

    title = doc.add_heading("Конспект лекции\n", level=1)
    title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    bullet_list = doc.add_paragraph()
    term_counter = 1

    for item in data:
        if isinstance(item, str):
            # Add a string as regular text with an interval
            if item.endswith('.'):
                item = item[:-1]
                
            item = item.split('\n')[0].capitalize()

            bullet_list.add_run(f"{item}. Основные термины:\n\n")
        else:
            # skip filename
            _, term, definition, timestart, timeend = item
            term = term.capitalize()
            definition = definition.capitalize()
            timestart_str = convert_to_hh_mm_ss(timestart)
            timeend_str = convert_to_hh_mm_ss(timeend)
            
            # Add an element to the bullet list with formatted content
            bullet_list.add_run(f"{term_counter}. {term} ").bold = True
            bullet_list.add_run(f"({timestart_str} - {timeend_str})\n{definition}").bold = False

            term_counter += 1

            bullet_list.add_run("\n\n")  # Add a line break

    return doc


In [ ]:
lecture_save_path = "lecture_final_run.docx"

doc = create_docx_from_list(list_items_for_docx)
doc.save(lecture_save_path)

In [25]:
def hack_pipeline(path):
    definitions = []
    
    transcript = whisper_transcribe_file(whisper_model, path_to_mp3)
    lecture_text = transcript['text']

    terms = get_terms_candidates(lecture_text, strong=True)
    cleared_terms = filter_terms_by_frequency(terms)
    final_terms = finalize_terms(terms, cleared_terms)
    final_terms_unique = cleared_terms.keys()

    timecodes, context = extract_timecodes_and_context(transcript, final_terms_unique)
    for prompt in get_rag_prompts(context):
        definitions.append(get_answer(model, tokenizer, prompt, DEVICE))
        
    return definitions

In [255]:
for path in paths:
    print(hack_pipeline(path))

In [44]:
! nvidia-smi

Fri Nov 24 17:03:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:8C:00.0 Off |                  Off |
| N/A   33C    P0    38W / 250W |  19500MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------